In [48]:
import numpy as np
import pandas as pd

s = pd.Series([
    4000.00,
    4000.50,
    4000.75,
    4001.50,
    4002.00,
    4005.00
])
s2 = pd.Series([
    4005.00,
    4002.00,
    4000.00,
    4001.50,
    4000.75,
    4000.50,
    3998.00
])
s3 = pd.Series([  # inverse W
    4000.00,
    4001.50,
    4002.00,
    4002.50,
    4001.50,
    4000.75,
    4000.00,
    3999.00,
    4000.00,
    4001.50,
    4003.00
])
s4 = pd.Series([  # hacksaw
    4000.00,
    4001.50,
    4000.00,
    4001.50,
    4000.00,
    4001.75,
    4000.25,
    # 4001.25
])


In [49]:
from typing import Iterable

class Renko:
    def __init__(self, s: pd.Series, bar_size: float):
        assert len(s) > 1, "Series of 2+ numbers needed"
        self.s = s
        self.size = bar_size

    def __iter__(self):
        # Assume initial trend with first two values
        if self.s[1] > self.s[0]:
            _hi_bound = self.s[0] + self.size
            _lo_bound = self.s[0] - (2 * self.size)
        else:
            _lo_bound = self.s[0] - self.size
            _hi_bound = self.s[0] + (2 * self.size)

        for i, x in self.s.iloc[1:].items():
            while x >= _hi_bound:
                yield (i, _hi_bound - self.size, _hi_bound, _hi_bound - self.size, _hi_bound)
                _hi_bound += self.size
                _lo_bound += self.size
            while x <= _lo_bound:
                yield (i, _lo_bound + self.size, _lo_bound + self.size, _lo_bound, _lo_bound)
                _hi_bound -= self.size
                _lo_bound -= self.size


renkos = list(Renko(s4, bar_size=0.75))
for x in renkos:
    print(x)

renkos = np.array(renkos)
renkos = pd.DataFrame(
    {
        "Open": renkos[:, 1],
        "High": renkos[:, 2],
        "Low": renkos[:, 3],
        "Close": renkos[:, 4],
    },
    # index=renkos[:, 0]
)

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Candlestick(
        # x=presentable.index,
        open=renkos["Open"],
        high=renkos['High'],
        low=renkos['Low'],
        close=renkos["Close"]
    )
])
fig.show()

(1, 4000.0, 4000.75, 4000.0, 4000.75)
(1, 4000.75, 4001.5, 4000.75, 4001.5)
(1, 4001.5, 4002.25, 4001.5, 4002.25)
(7, 4001.5, 4001.5, 4000.75, 4000.75)
(7, 4000.75, 4000.75, 4000.0, 4000.0)
(7, 4000.0, 4000.0, 3999.25, 3999.25)


In [26]:
esh2 = pd.read_parquet("/Users/beli/src/research/artifacts/ES_100V/ES-H2022_100V.parquet")

ts = esh2["Timestamp"]
px = round(((esh2["High"] + esh2["Low"]) / 2) / 0.25) * 0.25
px.index = ts
ra = np.array(list(Renko(px, 0.75)))
r = pd.DataFrame(
    {
        "Open": ra[:, 1],
        "High": ra[:, 2],
        "Low": ra[:, 3],
        "Close": ra[:, 4],
    },
    index=ra[:, 0]
)


In [27]:
import plotly.graph_objects as go

n_presentable = 1000
start_i = np.random.randint(0, r.shape[0] - n_presentable)
presentable = r.iloc[start_i:start_i + n_presentable]
print(f"Presenting from {start_i} to {start_i + n_presentable}")

fig = go.Figure(data=[
    go.Candlestick(
        # x=presentable.index,
        open=presentable["Open"],
        high=presentable['High'],
        low=presentable['Low'],
        close=presentable["Close"]
    )
])
fig.show()

Presenting from 77221 to 78221
